In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import re

In [2]:
from scipy.stats import skew,kurtosis

In [3]:
from scipy.special import cbrt

In [4]:
os.chdir('../output_data')

In [5]:
df = pd.read_csv('train.csv',header=0)

In [6]:
df.set_index('PassengerId',inplace=True)

In [7]:
df['Cabin']=df['Cabin'].astype(str).apply(lambda x: ''.join([s for s in x if str.isalpha(s)][0]))

In [8]:
df['Ticket']=df['Ticket'].apply(lambda x: ''.join([s for s in x if str.isalpha(s)]))

In [9]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

In [10]:
df['Pclass'] = df['Pclass'].astype(str).apply(lambda x: x+' class')

In [11]:
df['Title'] = df['Name'].apply(get_title) 

In [12]:
df.drop(['Name'],axis=1,inplace=True)

In [13]:
numerical,categorical = [],[]
for col in df.drop(['Survived'],axis=1).columns:
    if 'int' in str(df[col].dtypes) or 'float' in str(df[col].dtypes):
        numerical.append(col)
    else:
        categorical.append(col)

In [23]:
categorical.remove('Survived')

In [24]:
os.chdir('../pickles')

In [16]:
pickle.dump(numerical,open('numerical.pickle','wb'))

In [25]:
pickle.dump(categorical,open('categorical.pickle','wb'))

In [18]:
def edd(data):
    df_desc = data.describe().transpose()
    df_desc['Var'] = df_desc.index
    df_desc.reset_index(inplace=True)
    df_desc.drop('count',axis=1,inplace=True)
    df_desc['skewness'] = df_desc['Var'].apply(lambda x: skew(np.array(data.loc[data[x].notnull(),x])))
    df_desc['kurtosis'] = df_desc['Var'].apply(lambda x: kurtosis(np.array(data.loc[data[x].notnull(),x]),fisher=False))
    df_desc['99%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.99))
    df_desc['95%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.95))
    df_desc['90%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.90))
    df_desc['10%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.1))
    df_desc['5%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.05))
    df_desc['1%'] = df_desc['Var'].apply(lambda x: data[x].quantile(.01))
    df_desc['mean-3sigma'] = df_desc['mean'] - 3*df_desc['std']
    df_desc['mean+3sigma'] = df_desc['mean'] + 3*df_desc['std']
    df_desc['mean-2sigma'] = df_desc['mean'] - 2*df_desc['std']
    df_desc['mean+2sigma'] = df_desc['mean'] + 2*df_desc['std']
    df_desc['type']='numeric'
    
    def is_category(col):
        return 'float' not in str(data[col].dtype) and 'int' not in str(data[col].dtype)
    
    categorical = [col for col in data.columns if is_category(col)]
    df_categorical = pd.DataFrame()
    df_categorical['Var']=np.array(categorical)
    
    df_categorical['type']='categorical'
    for col in [c for c in df_desc.columns if c not in ['Var','type']]:
        df_categorical[col]=np.nan
    for col in categorical:
        df_var = data[col].value_counts()
        df_cat = pd.DataFrame()
        df_cat['count']=df_var
        df_cat['categories']=df_var.index
        df_cat.reset_index(inplace=True)
        df_cat.sort_values(by='count',ascending=False,inplace=True)
        df_cat.set_index('categories',inplace=True)
        index_list = df_cat.index.tolist()
        for i,c in enumerate(['mean','min','1%','5%','10%','25%']):
            try:
                df_categorical.loc[df_categorical['Var']==col,c] = index_list[i]
            except:
                break
        for i,c in enumerate(['50%','75%','90%','95%','99%','max']):
            try:
                df_categorical.loc[df_categorical['Var']==col,c] = index_list[-(i+1)]
            except:
                break
        del df_var
        del df_cat
        del index_list
    df_categorical = df_categorical[df_desc.columns]
    edd = pd.concat([df_desc,df_categorical])
    del df_desc
    del df_categorical
    edd['count'] = edd['Var'].apply(lambda x: data[data[x].notnull()].shape[0])
    edd['nmiss'] = data.shape[0]-edd['count']
    edd['missing_rate'] = np.array(edd['nmiss']).astype('float')/data.shape[0] * 100
    edd['unique'] = edd['Var'].apply(lambda x: len(data[x].value_counts().index.tolist()))
    orig_cols = ['mean','min','1%','5%','10%','25%','50%','75%','90%','95%','99%','max']
    new_cols = ['mean_or_top1','min_or_top2','p1_or_top3','p5_or_top4','p10_or_top5','p25_or_top6',
                'p50_or_bottom6','p75_or_bottom5','p90_or_bottom4','p95_or_bottom3','p99_or_bottom2','max_or_bottom1']
    
    convert_dict = {}
    for i in range(len(orig_cols)):
        convert_dict[orig_cols[i]]=new_cols[i]
    edd.rename(columns=convert_dict,inplace=True)
    edd = edd[['Var','type','count','nmiss','missing_rate','unique','std','skewness','kurtosis','mean-3sigma',
               'mean-2sigma','mean_or_top1','min_or_top2','p1_or_top3','p5_or_top4','p10_or_top5','p25_or_top6',
               'p50_or_bottom6','p75_or_bottom5','p90_or_bottom4','p95_or_bottom3','p99_or_bottom2','max_or_bottom1'
              ,'mean+2sigma','mean+3sigma']]
    return edd

In [19]:
edd = edd(df)

In [20]:
os.chdir('../Statistics')

In [21]:
edd.to_csv('edd_v01.csv',index=False)

In [49]:
from scipy.stats import chisquare
from statsmodels.stats.weightstats import ztest

In [50]:
def p_value_stat(col,target='Survived'):
    if col==target:
        return None
    try:
        if edd.loc[edd['Var']==col,'type'].values[0]=='categorical':
            obs_freq = []
            for cat in df[col].value_counts().index.tolist():
                obs_class=[]
                for i in df[target].value_counts().index.tolist():
                    obs = df[(df[col]==str(cat)) & (df[target]==i)].shape[0]
                    obs_class.append(obs)
                obs_freq.append(obs_class)
            obs_freq=np.array(obs_freq)
            return chisquare(obs_freq,axis=0)[1][0]
        else:
            return ztest(np.array(df.loc[(df[col].notnull()) & (df[target]==1),col]),
                        np.array(df.loc[(df[col].notnull()) & (df[target]==0),col]))[1]
    except Exception as e:
        print(e,col)
        return np.nan

In [56]:
edd['Anova/Chisquare pvalue'] = edd['Var'].apply(lambda x: p_value_stat(x))

In [57]:
transform_dict = {'log':lambda x: np.log(x),'sqr':lambda x: x**2,'sqrt':lambda x: np.sqrt(x),'exp':lambda x:np.exp(x),
                 'cube':lambda x: x**3,'cuberoot': lambda x: cbrt(x)}

In [58]:
edd['Status'] = ''

In [59]:
edd['transformed_p_value'] = np.nan

In [62]:
for col in [x for x in numerical if x != 'Survived']:
    p_max = edd.loc[edd['Var']==col,'Anova/Chisquare pvalue'].values
    if edd.loc[edd['Var']==col,'skewness'].values < 0:
        func_list = ['sqr','cube','exp']
    elif edd.loc[edd['Var']==col,'skewness'].values > 0:
        func_list = ['cuberoot','sqrt','log']
    for t in func_list:
        df_new = df[[col,'Survived']]
        df_new.dropna(how='any',axis=0,inplace=True)
        try:
            df_new[col] = df_new[col].apply(transform_dict[t])
            if not (True in np.isinf(np.array(df_new[col]))):
                p_value = ztest(np.array(df.loc[(df[col].notnull()) & (df['Survived']==1),col]),
                                np.array(df.loc[(df[col].notnull()) & (df['Survived']==0),col]))[1]
                if p_value<p_max:
                    p_max=p_value
                    edd.loc[edd['Var']==col,'Status']=t
                    edd.loc[edd['Var']==col,'Anova/Chisquare pvalue']=p_value
        except Exception as e:
            print(e)
            print(col)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [63]:
edd['Status'].value_counts()

    11
Name: Status, dtype: int64

In [65]:
edd.reset_index(inplace=True)

In [68]:
edd.drop(['index'],axis=1,inplace=True)

In [75]:
edd.to_csv('edd_v02.csv',index=False)

In [76]:
os.chdir('../output_data')

In [77]:
def transform(x,function):
    if x is not None:
        return function(x)
    else:
        return np.nan

In [78]:
for col in numerical:
    if edd.loc[edd['Var']==col,'Status'].values[0] != '':
        function = transform_dict[edd.loc[edd['Var']==col,'Status'].values[0]]
        df[col] = df[col].apply(lambda x: transform(x,function))

In [79]:
df.to_csv('train_v01.csv')